In [1]:
import pandas as pd

In [2]:

df = pd.read_csv('data/books_summary.csv')

In [3]:
df.head()

,Unnamed: 0,book_name,summaries,categories
0,0,The Highly Sensitive Person,is a self-assessment guide and how-to-live te...,science
1,1,Why Has Nobody Told Me This Before?,is a collection of a clinical psychologist’s ...,science
2,2,The Midnight Library,"tells the story of Nora, a depressed woman in...",science
3,3,Brave New World,presents a futuristic society engineered perf...,science
4,4,1984,is the story of a man questioning the system ...,science


In [4]:
import os
from openai import OpenAI
from dotenv import load_dotenv
client = OpenAI()
# 🔑 환경 변수 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [5]:
def embed_text(text):
    response = client.embeddings.create(
        model="text-embedding-ada-002",  # 임베딩 모델
        input=text
    )
    return response.data[0].embedding

In [6]:
!pip install tqdm


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import time
from tqdm import tqdm  # 진행률 표시 라이브러리

# 줄거리를 임베딩하고 새로운 컬럼에 저장
def embed_summaries(df):
    embeddings = []
    
    # tqdm을 사용하여 진행 상황 표시
    for summary in tqdm(df['summaries'], desc="임베딩 진행 중"):
        try:
            embedding = embed_text(summary)
            embeddings.append(embedding)
            time.sleep(0.5)  # OpenAI API 과부하 방지
        except Exception as e:
            print(f"임베딩 실패: {e}")
            embeddings.append(None)
    
    df['embedding'] = embeddings
    return df

# 실행
df = embed_summaries(df)


임베딩 진행 중:  16%|█▌        | 823/5201 [12:18<1:04:16,  1.14it/s]

임베딩 실패: Out of range float values are not JSON compliant: nan


임베딩 진행 중:  24%|██▎       | 1230/5201 [18:23<57:31,  1.15it/s]  

임베딩 실패: Out of range float values are not JSON compliant: nan


임베딩 진행 중:  36%|███▌      | 1854/5201 [27:33<48:27,  1.15it/s]  

...

임베딩 진행 중:  93%|█████████▎| 4828/5201 [1:12:17<05:23,  1.15it/s]

임베딩 실패: Out of range float values are not JSON compliant: nan


임베딩 진행 중: 100%|█████████▉| 5188/5201 [1:17:40<00:11,  1.11it/s]

임베딩 실패: Out of range float values are not JSON compliant: nan


임베딩 진행 중: 100%|██████████| 5201/5201 [1:17:50<00:00,  1.11it/s]


In [8]:
df.head()

,Unnamed: 0,book_name,summaries,categories,embedding
0,0,The Highly Sensitive Person,is a self-assessment guide and how-to-live te...,science,"[-0.027592381462454796, 0.02508273534476757, 0..."
1,1,Why Has Nobody Told Me This Before?,is a collection of a clinical psychologist’s ...,science,"[-0.010329793207347393, 0.0206208024173975, 0...."
2,2,The Midnight Library,"tells the story of Nora, a depressed woman in...",science,"[-0.01274832896888256, -0.017568137496709824, ..."
3,3,Brave New World,presents a futuristic society engineered perf...,science,"[-0.00034115940798074007, -0.00196639355272054..."
4,4,1984,is the story of a man questioning the system ...,science,"[0.015045898035168648, -0.02341669239103794, -..."


In [9]:
#  결과 저장 (중복 실행 방지)
import pickle
with open('embeddings.pkl', 'wb') as f:
    pickle.dump(df, f)
print("임베딩 결과가 'embeddings.pkl' 파일에 저장되었습니다.")

임베딩 결과가 'embeddings.pkl' 파일에 저장되었습니다.


In [10]:
#  CSV 저장 (외부 프로그램과 공유 가능)
df.to_csv('embeddings.csv', index=False)

In [16]:
import numpy as np
import pandas as pd

#  결측치(NaN 또는 None) 확인
missing_embeddings = df['embedding'].isnull().sum()
print(f"결측치(NaN 또는 None) 임베딩 개수: {missing_embeddings}")

#  임베딩 값이 올바른지(리스트인지) 확인
invalid_embeddings = df['embedding'].apply(lambda x: not isinstance(x, list)).sum()
print(f"리스트 형태가 아닌 임베딩 개수: {invalid_embeddings}")

#  임베딩 벡터 길이(차원) 검사 (예: 1536차원)
incorrect_dim_embeddings = df['embedding'].apply(lambda x: isinstance(x, list) and len(x) != 1536).sum()
print(f"1536차원이 아닌 임베딩 개수: {incorrect_dim_embeddings}")


결측치(NaN 또는 None) 임베딩 개수: 7
리스트 형태가 아닌 임베딩 개수: 7
1536차원이 아닌 임베딩 개수: 0


In [17]:
#  결측치(NaN 또는 None)인 데이터 출력
print("결측치(NaN 또는 None) 데이터:")
print(df[df['embedding'].isnull()])

#  리스트 형태가 아닌 데이터 출력
print("리스트 형태가 아닌 데이터:")
print(df[df['embedding'].apply(lambda x: not isinstance(x, list))])


결측치(NaN 또는 None) 데이터:
      Unnamed: 0                     book_name summaries     categories  \
823          829  The Power Of Full Engagement       NaN  relationships   
1230        1240  The Power Of Full Engagement       NaN      happiness   
1854        1869  The Power Of Full Engagement       NaN   productivity   
2433        2450  The Power Of Full Engagement       NaN     psychology   
3424        3449  The Power Of Full Engagement       NaN         health   
4828        4868  The Power Of Full Engagement       NaN           work   
5188        5232  The Power Of Full Engagement       NaN    mindfulness   

     embedding  
823       None  
1230      None  
1854      None  
2433      None  
3424      None  
4828      None  
5188      None  
리스트 형태가 아닌 데이터:
      Unnamed: 0                     book_name summaries     categories  \
823          829  The Power Of Full Engagement       NaN  relationships   
1230        1240  The Power Of Full Engagement       NaN      happiness   


In [1]:
import pickle

# pkl 파일 불러오기
with open('embeddings.pkl', 'rb') as f:
    df = pickle.load(f)

# 데이터 확인
print(df.head())


   Unnamed: 0                            book_name  \
0           0          The Highly Sensitive Person   
1           1  Why Has Nobody Told Me This Before?   
2           2                 The Midnight Library   
3           3                      Brave New World   
4           4                                 1984   

                                           summaries categories  \
0   is a self-assessment guide and how-to-live te...    science   
1   is a collection of a clinical psychologist’s ...    science   
2   tells the story of Nora, a depressed woman in...    science   
3   presents a futuristic society engineered perf...    science   
4   is the story of a man questioning the system ...    science   

                                           embedding  
0  [-0.027592381462454796, 0.02508273534476757, 0...  
1  [-0.010329793207347393, 0.0206208024173975, 0....  
2  [-0.01274832896888256, -0.017568137496709824, ...  
3  [-0.00034115940798074007, -0.00196639355272054...

In [2]:
# 임베딩이 리스트인지 확인
is_list = df['embedding'].apply(lambda x: isinstance(x, list)).all()
print(f"모든 임베딩이 리스트 형태인가요? {is_list}")


모든 임베딩이 리스트 형태인가요? False


In [4]:
import ast

# 문자열로 저장된 임베딩을 리스트로 변환
df['embedding'] = df['embedding'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# 변환 후 확인
is_list = df['embedding'].apply(lambda x: isinstance(x, list)).all()
print(f"모든 임베딩이 리스트 형태인가요? {is_list}")


모든 임베딩이 리스트 형태인가요? False


In [3]:
# summaries 컬럼에서 결측치(NaN 또는 None) 제거
df = df[df['summaries'].notnull()]

# embedding 컬럼에서도 결측치(NaN 또는 None) 제거
df = df[df['embedding'].notnull()]

# 결과 확인
print(f"남은 데이터 개수: {len(df)}")


남은 데이터 개수: 5194


In [4]:
# 임베딩이 리스트인지 확인
is_list = df['embedding'].apply(lambda x: isinstance(x, list)).all()
print(f"모든 임베딩이 리스트 형태인가요? {is_list}")


모든 임베딩이 리스트 형태인가요? True


In [5]:
# 임베딩 벡터의 길이 확인
embedding_dim_check = df['embedding'].apply(lambda x: len(x) == 1536 if isinstance(x, list) else False).all()
print(f"모든 임베딩 벡터가 1536 차원인가요? {embedding_dim_check}")


모든 임베딩 벡터가 1536 차원인가요? True


In [11]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
# 유사한 책 검색 함수
def search_similar_books(book_title, top_n=5):
    try:
        # 입력한 책 제목의 임베딩 확인
        embedding_check = df.loc[df['book_name'] == book_title_input, 'embedding']
        print(f"임베딩 확인: {embedding_check}")
        # 입력된 책 제목과 일치하는 책의 임베딩 불러오기
        query_embedding = df.loc[df['book_name'] == book_title, 'embedding'].values[0]

        # 책 임베딩 불러오기 (이미 저장된 임베딩 사용)
        book_embeddings = np.array(df['embedding'].tolist())

        # 코사인 유사도 계산
        similarities = cosine_similarity([query_embedding], book_embeddings)[0]

        # 유사도가 높은 Top N 도서 인덱스 추출 (자기 자신 제외)
        top_indices = similarities.argsort()[-(top_n + 1):][::-1]  # 자기 자신 제외를 위해 +1
        top_indices = [idx for idx in top_indices if df.iloc[idx]['book_name'] != book_title][:top_n]

        # 추천 결과 반환 (책 제목, 줄거리, 카테고리)
        return df.iloc[top_indices][['book_name', 'summaries', 'categories']]

    except IndexError:
        print(f"'{book_title}' 제목의 책을 찾을 수 없습니다.")
        return None
    except Exception as e:
        print(f"유사도 계산 중 오류 발생: {e}")
        return None

In [12]:

# 예시 실행
book_title_input = input("추천을 원하는 책 제목을 입력하세요: ")
recommended_books = search_similar_books(book_title_input, top_n=5)

# 결과 확인
if recommended_books is not None:
    print(recommended_books)


Empty DataFrame
Columns: [book_name, summaries, categories]
Index: []
